In [2]:
import pandas as pd
import arcpy
import numpy as np
import os
from arcpy import management

arcpy.env.overwriteOutput = True
arcpy.env.workspace = r"P:\Temp\McAvoy\ML_DataCollection\MeridianHills\\"

In [43]:
points = r"P:\Temp\McAvoy\ML_DataCollection\MeridianHills\MeridianHills.xlsx"
sheet = "MeridianHills"
dem = r"C:\Users\smcavoy\Downloads\USGS_13_n40w087.tif"
parcels = r"P:\Temp\McAvoy\ML_DataCollection\MeridianHills\MeridianHills_DataCollection\MeridianHills_DataCollection.gdb\Parcels_Merge.shp"
nfhl_flood_hazard_area = r"P:\Temp\McAvoy\ML_DataCollection\MeridianHills\MeridianHills_DataCollection\NFHL_18_20200310.gdb\S_FLD_HAZ_AR.shp"
frd_depth_grid_100yr = ""
frd_depth_grid_10yr = ""
building_footprints = r"P:\Temp\McAvoy\ML_DataCollection\MeridianHills\MeridianHills_DataCollection\MeridianHills_DataCollection.gdb\Indiana_JSONToFeatures.shp"
land_use = r"P:\Temp\McAvoy\ML_DataCollection\NLCD_2016_Land_Cover_L48_20190424.img"
subwatershed = r"P:\Temp\McAvoy\ML_DataCollection\MeridianHills\MeridianHills_DataCollection\MeridianHills_DataCollection.gdb\HUC051202011006"

In [ ]:
#turn Milliman point csv into points shapefile 
points = arcpy.ExcelToTable_conversion(points, "points", sheet)

arcpy.management.XYTableToPoint(points, "milliman_points", "lon", "lat", "GroundElev")



In [31]:
points = r"P:\Temp\McAvoy\ML_DataCollection\MeridianHills\MeridianHills_DataCollection\MeridianHills_DataCollection.gdb\MeridianHills_Points"

In [5]:



#select building footprints that intersect with Milliman points
selection1 = arcpy.SelectLayerByLocation_management(building_footprints, "INTERSECT", points, 0, "NEW_SELECTION", "NOT_INVERT")

print("selected building footprints")

#copy selected features to new feature class
arcpy.CopyFeatures_management(selection1, "footprints")

print("copied building footprints")


selected building footprints
copied building footprints


In [16]:
#switch selection1 so that only points that DONT intersect building footprints are selected
selection2 = arcpy.SelectLayerByLocation_management(points, "INTERSECT", "footprints.shp", 0, "NEW_SELECTION", "INVERT")

print("selected points")

#copy the selected footprints to new feature class
extra_points = arcpy.CopyFeatures_management(selection2, "extra_points")

print("copied extra points")




selected points
copied extra points


In [17]:

#make a 10m buffer around the points that don't have footprints
buffer = arcpy.Buffer_analysis("extra_points.shp", "new_footprints", "10 Meters")

print("buffered points")

buffered points


In [18]:
#merge the selected Bing building footprints witht the newly created buffer footprints
all_footprints = arcpy.Merge_management(["footprints.shp", "new_footprints.shp"], "all_footprints")



In [19]:
#get max elevation on footprints
max_el_footprints = arcpy.sa.ZonalStatistics(all_footprints, "FID", arcpy.sa.Raster(dem), "MAXIMUM")

#get min elevation on footprints
min_el_footprints = arcpy.sa.ZonalStatistics(all_footprints, "FID", arcpy.sa.Raster(dem), "MINIMUM")

print("max and min elevations gotten for footprints")

max and min elevations gotten for footprints


In [20]:
#select parcels that intersect with milliman points
parcels_selection = arcpy.SelectLayerByLocation_management(parcels, "INTERSECT", points, 0, "NEW_SELECTION", "NOT_INVERT")
arcpy.CopyFeatures_management(parcels_selection, "all_parcels")

<Result 'P:\\Temp\\McAvoy\\ML_DataCollection\\MeridianHills\\all_parcels.shp'>

In [22]:
#get max elevation on parcels
max_el_parcels = arcpy.sa.ZonalStatistics("all_parcels.shp", "FID", arcpy.sa.Raster(dem), "MAXIMUM")

#get min elevation on parcels
min_el_parcels = arcpy.sa.ZonalStatistics("all_parcels.shp", "FID", arcpy.sa.Raster(dem), "MINIMUM")

#get avg elevation on parcels
avg_el_parcels = arcpy.sa.ZonalStatistics("all_parcels.shp", "FID", arcpy.sa.Raster(dem), "MEAN")

print("max, min, and average elevations gotten for parcels")

max, min, and average elevations gotten for parcels


In [23]:
#get slope of all pixels in the dem
slope = arcpy.sa.Slope(arcpy.sa.Raster(dem), "PERCENT_RISE")

#get average slope for parcels
slope_parcels = arcpy.sa.ZonalStatistics("all_parcels.shp", "FID", arcpy.sa.Raster(slope), "MEAN")

print("slope gotten for all parcels")

slope gotten for all parcels


In [25]:
#get aspect of all pixels in the dem
aspect = arcpy.sa.Aspect(arcpy.sa.Raster(dem))

#get average aspect for parcels
aspect_parcels = arcpy.sa.ZonalStatistics("all_parcels.shp", "FID", arcpy.sa.Raster(aspect), "MEAN")

print("aspect gotten for all parcels")


aspect gotten for all parcels


In [36]:
#get distance to flooding source
#use Near tool to create XY location of nearest point and distance
#distance between milliman point and NFHL S_FLD_HAZ_AR where Flood Zone = AE and Zone Subtype = Floodway

#select floodways and make new feature class
floodway = arcpy.management.SelectLayerByAttribute(nfhl_flood_hazard_area, "NEW_SELECTION", 
                                                   "FLD_ZONE = 'AE' And ZONE_SUBTY = 'FLOODWAY'", None)

arcpy.CopyFeatures_management(floodway, "nfhl_floodway")

print("created nfhl floodway shapefile")

#use Near tool, specify LOCATION to include XY coordinates along floodways 
arcpy.Near_analysis(points, "nfhl_floodway.shp", "", "LOCATION" )

print("ran Near tool")


created nfhl floodway shapefile
ran Near tool


In [37]:
#get 100 yr and 10 yr flood depths at nearest points in floodways 
#make points from XY data in Near table output
near_points = arcpy.XYTableToPoint_management(points, "near_points", "NEAR_X", "NEAR_Y")

print("Near points created")

#extract values from FRD
raster_list1 = [[frd_depth_grid_100yr, "depth100_riv"], [frd_depth_grid_10yr, "depth10_riv"]]
arcpy.sa.ExtractMultiValuesToPoints(near_points, raster_list1)

print("extracted flood depths at near points")

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

Near points created


ExecuteError: ERROR 001268: Input rasters: The input item, , is invalid.
Failed to execute (ExtractMultiValuesToPoints).


In [40]:
#get the values from all of the rasters created and collected extracted to points
raster_list2 = [[max_el_footprints, "max_fp"], [min_el_footprints, "min_fp"], [max_el_parcels, "max_parcel"],
               [min_el_parcels, "min_parcel"], [avg_el_parcels, "avg_parcel"], [slope_parcels, "slope"], 
                [aspect_parcels, "aspect"],  [arcpy.sa.Raster(land_use), "land_use"]]
#[frd_depth_grid_100yr, "depth100"],  [frd_depth_grid_10yr, "depth10"]]

arcpy.sa.ExtractMultiValuesToPoints(points, raster_list2)


<geoprocessing server result object at 0x1df12aa4648>

In [41]:
#join the near_points features with the points features based on the plus code
#arcpy.AddJoin_management(points, "plus_code", near_points, "plus_code")

#spatial join the tax parcel fields that have to do with property value
arcpy.SpatialJoin_analysis(points, parcels, "final_points", "JOIN_ONE_TO_MANY", "KEEP ALL", )


ExecuteError: ERROR 000622: Failed to execute (Spatial Join). Parameters are not valid.
ERROR 000800: The value is not a member of KEEP_ALL | KEEP_COMMON.


## subwatershed (HUC12) characteristics ##

In [45]:
#calculate area of subwatershed
arcpy.AddField_management(subwatershed, )
area = arcpy.CalculateGeometryAttributes_management(subwatershed, "AREA", "", "SQUARE_MILES_US")
print(area)

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

RuntimeError: Cannot find field 'AREA'

ExecuteError: ERROR 000582: Error occurred during execution.


In [ ]:
#calculate perimeter of subwatershed
perimeter = arcpy.CalculateGeometryAttributes_management

In [ ]:
#calculate watershed length
#distance from outlet to watershed boundary along the main channel
